# Forward dPL Multi-layer Terzaghi's 1D Consolidation Model

-- Land subsidence project: 15 March 2025 --

---


#### 1. Data Preperations

Essentially, the internals of what will eventually be the data loader for this model.

In [1]:
### Load in data
import sys
import os
import numpy as np
import torch
import pickle

sys.path.append('../../')
sys.path.append('../../dMG')  # Add the dMG root directory.
sys.path.append(os.path.abspath('..'))  # Add the parent directory of `scripts` to the path.

from scripts import load_config


#------------------------------------------#
# Define model settings here.
CONFIG_PATH = '/projects/mhpi/leoglonz/dPLT/src/dMG/conf/config_ls.yaml'
TEST_SPLIT = 0.2
#------------------------------------------#



config = load_config(CONFIG_PATH)


# Load data
with open(config['observations']['train_path'], 'rb') as f:
    data_dict = pickle.load(f)


# Normalize attributes for NN:
attrs = data_dict['attributes']
for i in range(attrs.shape[-1]):
    attrs[:, :, i] = (attrs[:, :, i] - attrs[:, :, i].mean()) \
        / attrs[:, :, i].std()
    
data_dict['xc_nn_norm'] = attrs


# Train-test split + convert to torch tensors
train_dataset = {}
test_dataset = {}
split = int(len(data_dict['forcing']) * (1 - TEST_SPLIT))

for key in data_dict.keys():
    train_dataset[key] = torch.tensor(
        data_dict[key][:split,],
        dtype=config['dtype'],
        device=config['device'],
    )
    test_dataset[key] = torch.tensor(
        data_dict[key][split:,],
        dtype=config['dtype'],
        device=config['device'],
    )

# Reshape to 3d
shape = train_dataset['xc_nn_norm'].shape
train_dataset['xc_nn_norm'] = train_dataset['xc_nn_norm'].reshape(
    shape[0], 
    shape[1],
    shape[2] * shape[3],
)

shape = test_dataset['xc_nn_norm'].shape
test_dataset['xc_nn_norm'] = test_dataset['xc_nn_norm'].reshape(
    shape[0], 
    shape[1],
    shape[2] * shape[3],
)   

HydroDL2 not found. Continuing without it.


#### 2. Load dPL Scheme 

Physical model (Terzaghi's equation) + LSTM


In [2]:
import importlib

from dMG import load_nn_model
from src.dMG.models.phy_models.terzaghi import TerzaghiMultiLayer as dPLT
importlib.reload(sys.modules['src.dMG.models.phy_models.terzaghi'])
importlib.reload(sys.modules['dMG'])


model = dPLT(config['dpl_model']['phy_model'], device=config['device'])
nn = load_nn_model(
    model,
    config['dpl_model'],
    device=config['device'],
)

#### 3. Forward Model


In [3]:
parameters = nn(train_dataset['xc_nn_norm'])

print(nn)
print(f"Input shape {train_dataset['xc_nn_norm'].shape}")
print(f"Output shape {parameters.shape}")

# predictions = model(
#     train_dataset,
#     parameters,
# )

LstmModel(
  (linear_in): Linear(in_features=81, out_features=256, bias=True)
  (lstm): Lstm(
    (lstm): LSTM(256, 256)
  )
  (linear_out): Linear(in_features=256, out_features=1, bias=True)
)
Input shape torch.Size([109, 16, 81])
Output shape torch.Size([109, 16, 1])


/projects/mhpi/leoglonz/dPLT/.venv/lib/python3.12/site-packages/torch/nn/modules/rnn.py:1124: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at /pytorch/aten/src/ATen/native/cudnn/RNN.cpp:1412.)
  result = _VF.lstm(


In [21]:
parameters.shape

torch.Size([109, 16, 8])

In [4]:
train_dataset['attributes'].shape

torch.Size([109, 16, 9, 9])

suppose i have raw_parameters = x_dict['attributes'] with shape [time, sites, layers, parameters, 16] (repeated 16 times). I have another set of parameters which are learned from an lstm, parameters, but this only contains a certain set of parameters learned for a certain set of layers. For example, if i learned the first two parameters for layers 0 and 1, this would look like parameters ~ raw_parameters[time, sites, :2, :2, 16], where the 5th dimension is 16 because I learn the parameter 16 different times and take the average to reduce variance. This is represented by self.nmul in my code. Knowing this, can you modify the parameter handling functions in my model class to overwrite raw_parameters with parameters?